In [1]:
import numpy as np
import numpy.linalg
from tabulate import tabulate

In [2]:
def mgs(A):
    m = A.shape[0]
    n = A.shape[1]
    Q = np.zeros((m,n))
    R = np.zeros((n,n))
    
    for i in range(n):
        v = A[:,i]
        
        for j in range(i):
            R[j,i] = Q[:,j].T.dot(A[:,i])
            v = v.squeeze() - (R[j,i]*Q[:,j])
        R[i,i] = np.linalg.norm(v)
        Q[:,i] = (v / R[i,i]).squeeze()
    return Q, R

In [3]:
def house(A):
    m = A.shape[0]
    n = A.shape[1]
    R = A.copy()
    U = np.zeros((m,n))
    for i in range(n):
        x = R[i:m, i]
        e = np.zeros((1,len(x)))
        e[0][0] = 1
        u = np.sign(x[0]) * np.linalg.norm(x) * e + x
        u = u/np.linalg.norm(u)
        
        R[i:m, i:n] = R[i:m, i:n] - 2.0 * np.outer(u.T, np.dot(u, R[i:m, i:n]))
        U[i:m, i] = u.squeeze()
    R = np.delete(R, range(R.shape[1], R.shape[0]), axis = 0)
    return U, R

In [4]:
m, n = 50, 12

t = np.linspace(0,1,num = m)
A = np.fliplr(np.vander(t, n))

b = np.cos(4*t)

In [5]:
x1 = np.linalg.solve(A.T.dot(A), A.T.dot(b))

In [6]:
Q, R = mgs(A)
x2 = np.linalg.solve(R, Q.T.dot(b))

In [7]:
U, R = house(A)
Q = A.dot(np.linalg.inv(R))
x3 = np.linalg.solve(R, Q.T.dot(b))

In [8]:
Q, R = np.linalg.qr(A)
x4 = np.linalg.solve(R, Q.T.dot(b))

In [9]:
x5 = np.linalg.lstsq(A, b)[0]

In [10]:
U, s, V = np.linalg.svd(A)
S = np.zeros((m,n))
S[:s.shape[0], :s.shape[0]] = np.diag(s)

x6 = V.T.dot(np.linalg.lstsq(S, U.T.dot(b))[0])

In [11]:
table1 = {'Normal':x1, 'MGS':x2, 'House':x3}
table2 = {'QR':x4, 'x = A/b':x5, 'SVD':x6}
print tabulate(table1, headers = 'keys', floatfmt = '.16f')
print tabulate(table2, headers = 'keys', floatfmt = '.16f')

                 MGS                House               Normal
--------------------  -------------------  -------------------
  1.0000053533240936   1.0000000045326869   0.9999999821271303
 -0.0005758979056823  -0.0000014868986768   0.0000052756829734
 -7.9911572340157626  -7.9999409774704668  -8.0001973137839695
 -0.0194657675857019  -0.0009147724766076   0.0028870948110900
 10.3156828084165255  10.6740092065298011  10.6447553570670070
  2.7762759000009178  -0.0345351175477042   0.0980231857515497
-14.8146797608050509  -5.5882255741704423  -5.9653329205408774
 16.2256716152385039  -0.1832647656926375   0.5093129370633428
-14.6174823581323388   1.8213219629196709   1.0010238386447288
  8.5868958215562152  -0.0880833148096715   0.5170088167573208
 -2.1511844577205048  -0.3349538904796512  -0.5877712985575644
  0.0363558870945668   0.0809451034728088   0.1266414394081457
                SVD                   QR              x = A/b
-------------------  -------------------  --------------